In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,18200421985,13565782190,463613841,463613841,213294893,213294893,229638555,229638555,418040584,418040584
2,12833389832,12833389832,250468052,250468052,64060997,64060997,84341210,84341210,383708488,383708488
3,38911337285,14852658085,719455455,719455455,343823691,343823691,309254926,309254926,721374614,721374614
4,35520744282,15144713938,282404961,282404961,150756074,150756074,145188335,145188335,640802011,640802011


In [3]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 4
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [ ]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 56s 28ms/step - loss: 0.0159 - val_loss: 0.0134
Epoch 2/20
2000/2000 [==============================] - 52s 26ms/step - loss: 0.0119 - val_loss: 0.0131
Epoch 3/20
1378/2000 [===================>..........] - ETA: 36s - loss: 0.0113

In [ ]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[29134251061,187609472,194216024,110492056,439310873]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


In [ ]:
# Save trained model
model.save("model.h5")
